In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D 
from tensorflow.keras.datasets import cifar10

# 이미 훈련과 테스트가 분리된 데이터임
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [ ]:
# 데이터의 크기 확인
# 28X28X1 = 흑백데이터다~ 
print("train",x_train.shape,y_train.shape) #(50000, 32, 32, 3) (50000, 1)
print("test",x_test.shape,y_test.shape) # (10000, 32, 32, 3) (10000, 1)



In [ ]:
# y의 클래스 보기
a,b = np.unique(y_train,return_counts=1)
# print(np.unique(y_train,return_counts=1))
print(a)
print(b)

In [ ]:
# 크기변경
# x_train = x_train.reshape(50000, 32, 32, 3)
# x_test = x_test.reshape(50000, 32, 32, 3)

In [ ]:
# 이미지 시각화
import matplotlib.pyplot as plt
plt.imshow(x_train[1])

In [ ]:
# 모델
# 베이스모델불러오기
from tensorflow.keras.applications import efficientnet
def build_model(input_shape, n_classes):

    base_model = efficientnet.EfficientNetB0(input_shape=input_shape,
                                      weights="imagenet",
                                      include_top=False,
                                      classes=10)

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = build_model(input_shape=(32, 32, 3), n_classes=10)
model.summary()

In [ ]:

# model = Sequential([
#     # 5X5 이미지 채널 1개 
#     # 필터 = output 노드수 
#     # 자르는 필터는 2X2 사이즈 
#     Conv2D(filters=128,kernel_size=(2,2),input_shape=(32,32,3),activation="relu"),

#     Conv2D(filters=64,kernel_size=(2,2),padding="valid",activation="relu"),   
     
     
#     # 완전통합층에서는 falt + Flatten(),
#     # Dense 의 인풋은 batch_size , input_dim
#     Flatten(),
#     Dense(32,activation="relu"),
#     Dense(10,activation="softmax")
# ])

# model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["acc"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

earlyStopping = EarlyStopping(monitor="val_loss",
                              mode="min",patience =10,
                              # True 면 좋았던 웨이트로 복원됨
                              # False 면 마지막 웨이트로 놔둠..
                              restore_best_weights=True,
                              verbose=1                              
                              )
# 모델체크포인트는 가장 낮은 val_loss 가 갱신 될 떄 마다 저장합니다.

mcp = ModelCheckpoint(
    monitor="val_loss",
    mode="auto",
    save_best_only=True,
    filepath= "cifar.h5",
    verbose=1 
)


hist = model.fit(
                x_train,
                y_train,
                epochs=10,
                batch_size=64,
                validation_split=0.2,
                verbose=1,
                callbacks = [earlyStopping,mcp],      
                )

In [ ]:
loss,acc = model.evaluate(x_test,y_test)
print("loss : ",loss)
print("acc : ",acc)


In [ ]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['acc', 'val_acc'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()